# Select Customers to Participate the Loyalty Program of a Retailer Using Clustering Analysis

#### Author: Adrieli Dal'Toé

## Part II - Data Preparation

Now that the data is clean and have only the information under interest, I will perform a series of manipulations to extract features that might help to solve this business problem. The data was provided in a format not appropriate for this problem. Each row of the data set represents a product that was purchased in a given transaction by a specific customer. The goal of the project is to find the high-value customers to participate in the loyalty program the company wants to create. Thus, it is more important to know how many purchases the customer made, how much was spent in a certain period of time, what time of the year the purchases were made and in what frequency, rather than discriminate the items purchased in each transaction. 

This part of the project is composed by the following steps:
 - Feature engineering. What features are important to separate customers into distinct groups?
 - Outliers treatment. Outliers are extreme values that differentiate from the main patterns of the data.
 - Feature analysis and correlation
 - Standardizing Features. Clustering algorithms use distance as a mean to measure similarity. Different range of values in each feature will act as a weight determining how to cluster data, introducing bias into the model.

In [1]:
from platform import python_version
print('Python Version:', python_version())

Python Version: 3.8.8


### Libraries

In [3]:
# Data manipulation
import pandas as pd
import numpy as np
import re

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

# import modules
import sys
sys.path.append("lib")
from customize_figure import figure_layout

In [4]:
print('Package version')
print('Re: ', re.__version__)
print('Pandas: ', pd.__version__)
print('Numpy: ', np.__version__)
print('Seaborn: ', sns.__version__)
print('Plotly: ', plotly.__version__)

Package version
Re:  2.2.1
Pandas:  1.2.4
Numpy:  1.20.1
Seaborn:  0.11.1
Plotly:  5.1.0


### Import data

In [27]:
df = pd.read_csv('data/ecommerce_clean.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34


In [28]:
df.shape

(405490, 9)

In [29]:
df.describe()

,Quantity,UnitPrice,CustomerID,purchase
count,405490.000000,405490.000000,405490.000000,405490.000000
mean,12.093672,3.307532,15294.824440,20.292136
std,249.102968,66.640555,1710.359858,427.840161
min,-80995.000000,0.000000,12346.000000,-168469.600000
25%,2.000000,1.250000,13969.000000,4.200000
50%,5.000000,1.950000,15159.000000,10.750000
75%,12.000000,3.750000,16794.000000,19.500000
max,80995.000000,38970.000000,18287.000000,168469.600000


### Extracting year, month and day

In [30]:
df['date'] = pd.to_datetime(df['InvoiceDate'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchase,date,year,month,day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,15.30,2016-11-29,2016,11,29
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34,2016-11-29,2016,11,29
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,22.00,2016-11-29,2016,11,29
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34,2016-11-29,2016,11,29
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34,2016-11-29,2016,11,29


In [31]:
df['date'].max() - df['date'].min()

Timedelta('373 days 00:00:00')

I will use a time interval of 365 days (1 year) in this project. 

In [32]:
df.drop(index = df[df['date']>='2017-11-30 00:00:00'].index, inplace=True)

df['date'].max() - df['date'].min()

Timedelta('365 days 00:00:00')

### total_purchase = Quantity*UnitPrice

Total amount of money spent by customers within 1 year period.

In [34]:
df['total_purchase']=df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchase,date,year,month,day,total_purchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,15.30,2016-11-29,2016,11,29,15.30
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34,2016-11-29,2016,11,29,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,22.00,2016-11-29,2016,11,29,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34,2016-11-29,2016,11,29,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,20.34,2016-11-29,2016,11,29,20.34


## Feature Engineering

df_clusters dataframe will organize the data in the proper format for clustering analysis. New features will be created using the data available and domain knowledge.

### df_clusters dataframe

In [105]:
# Total purchase per customer
df_clusters = df.groupby('CustomerID')['total_purchase'].sum().round(2).reset_index()
df_clusters.describe()

,CustomerID,total_purchase
count,4334.000000,4334.000000
mean,15302.260960,1830.320295
std,1721.089586,7947.383176
min,12346.000000,-4287.630000
25%,13816.250000,288.465000
50%,15302.000000,630.265000
75%,16780.750000,1543.495000
max,18287.000000,266738.000000


As expected, total_purchase have negative values because some customers had just C_transaction in the time period the data was collected, as seen in the data cleaning step. However, negative values for total_purchase also indicate customers that made more returns than valid purchases. This is the case of CustomerID 17603 that placed the first valid order on 20-Sep-17, and returns were registered before this date clearly related to purchases placed before 29-Nov-2016. 

In [106]:
df[df['CustomerID']==17603.0].groupby('InvoiceNo').agg({'Quantity':sum,
                                                        'purchase':sum,
                                                        'InvoiceDate':lambda x:x.unique()})

,Quantity,purchase,InvoiceDate
InvoiceNo,,,
567868,74,394.72,20-Sep-17
C546859,-102,-555.90,15-Mar-17
C560435,-1,-294.92,16-Jul-17
C560436,-1,-705.45,16-Jul-17
C572117,-1,-3.75,18-Oct-17


In [107]:
df_clusters.shape

(4334, 2)

### Number of valid orders
I will remove all the C_transactions (Quantity<0) and the examples where UnitPrice = 0 to estimate the number of valid transactions.

In [108]:
df_valid_orders = df[(df['UnitPrice']>0)&(df['Quantity']>0)]

number_orders = df_valid_orders.groupby('CustomerID')['InvoiceNo'].nunique()\
                               .reset_index()\
                               .rename(columns={'InvoiceNo':'number_orders'})


df_clusters = df_clusters.merge(right=number_orders, how='left', on='CustomerID')
df_clusters.fillna(0, inplace = True)
df_clusters.shape

(4334, 3)

### Number of C_transactions

In [109]:
df[df['Quantity']<0]['InvoiceNo'].unique().shape

(3472,)

In [110]:
n_C_transactions = df[df['Quantity']<0].groupby('CustomerID')['InvoiceNo'].nunique()\
                                       .reset_index()\
                                       .rename(columns={'InvoiceNo':'number_returns'})

df_clusters = df_clusters.merge(right=n_C_transactions, 
                                on='CustomerID', 
                                how='left')

df_clusters.fillna(0, inplace=True)

df_clusters.head()

,CustomerID,total_purchase,number_orders,number_returns
0,12346.0,0.00,1.0,1.0
1,12347.0,4085.18,6.0,0.0
2,12348.0,1437.24,4.0,0.0
3,12349.0,1457.55,1.0,0.0
4,12350.0,294.40,1.0,0.0


In [111]:
df_clusters.shape

(4334, 4)

### Number of orders with products of UnitPrice = 0

In [112]:
orders_with_gifts = df[df['UnitPrice']==0].groupby('CustomerID')['InvoiceNo'].nunique()\
                                          .reset_index()\
                                          .rename(columns={'InvoiceNo':'orders_with_gifts'})
df_clusters = df_clusters.merge(right=orders_with_gifts, on='CustomerID', how='left')
df_clusters.fillna(0, inplace=True)
df_clusters.head()

,CustomerID,total_purchase,number_orders,number_returns,orders_with_gifts
0,12346.0,0.00,1.0,1.0,0.0
1,12347.0,4085.18,6.0,0.0,0.0
2,12348.0,1437.24,4.0,0.0,0.0
3,12349.0,1457.55,1.0,0.0,0.0
4,12350.0,294.40,1.0,0.0,0.0


### Number of orders per month

The average number of valid orders a customer has placed per month.

In [113]:
order_per_month = df_valid_orders.groupby(['CustomerID','month'])['InvoiceNo'].nunique()\
                                 .reset_index()\
                                 .groupby('CustomerID')['InvoiceNo'].mean()\
                                 .reset_index()\
                                 .rename(columns={'InvoiceNo':'order_per_month'})

df_clusters = df_clusters.merge(right=order_per_month, how='left', on='CustomerID')
df_clusters.head()

,CustomerID,total_purchase,number_orders,number_returns,orders_with_gifts,order_per_month
0,12346.0,0.00,1.0,1.0,0.0,1.0
1,12347.0,4085.18,6.0,0.0,0.0,1.0
2,12348.0,1437.24,4.0,0.0,0.0,1.0
3,12349.0,1457.55,1.0,0.0,0.0,1.0
4,12350.0,294.40,1.0,0.0,0.0,1.0


### Number of months with purchases

This feature counts the number of months a customer placed new orders.

In [114]:
months_with_order = df_valid_orders.groupby(['CustomerID'])['month'].nunique()\
                                   .reset_index()\
                                   .rename(columns={'month':'months_with_order'})

df_clusters = df_clusters.merge(right=months_with_order, how='left', on='CustomerID')
df_clusters.head()

,CustomerID,total_purchase,number_orders,number_returns,orders_with_gifts,order_per_month,months_with_order
0,12346.0,0.00,1.0,1.0,0.0,1.0,1.0
1,12347.0,4085.18,6.0,0.0,0.0,1.0,6.0
2,12348.0,1437.24,4.0,0.0,0.0,1.0,4.0
3,12349.0,1457.55,1.0,0.0,0.0,1.0,1.0
4,12350.0,294.40,1.0,0.0,0.0,1.0,1.0


### What time of the year customers have purchased?

Months are divided into four groups representing quarters of the year. These features will inform in which time of the year customers have purchased. 

In [118]:
# Quarter of the year
df['year_quarter'] = np.select(
    [
        df['month'].between(1, 3, inclusive=True), 
        df['month'].between(4, 6, inclusive=True),
        df['month'].between(7, 9, inclusive=True),
        df['month'].between(10, 12, inclusive=True)
    ], 
    [
        '1_3Month', 
        '4_6Month',
        '7_9Month',
        '10_12Month'
    ], 
    default='Unknown'
)

In [123]:
quarter_of_purchase = df.groupby(['CustomerID','InvoiceNo'])\
                                     .agg({'year_quarter': lambda x: x.unique()})\
                                     .reset_index()

quarter_number_puchases = pd.get_dummies(quarter_of_purchase, columns=['year_quarter'], prefix="", prefix_sep="")\
                            .drop(columns='InvoiceNo')\
                            .groupby('CustomerID')\
                            .sum()\
                            .reset_index()
quarter_number_puchases.head()

,CustomerID,10_12Month,1_3Month,4_6Month,7_9Month
0,12346.0,0,2,0,0
1,12347.0,2,1,2,1
2,12348.0,1,1,1,1
3,12349.0,1,0,0,0
4,12350.0,0,1,0,0


In [124]:
df_clusters = df_clusters.merge(right=quarter_number_puchases, how='left', on='CustomerID')
df_clusters.head()

,CustomerID,total_purchase,number_orders,number_returns,orders_with_gifts,order_per_month,months_with_order,10_12Month,1_3Month,4_6Month,7_9Month
0,12346.0,0.00,1.0,1.0,0.0,1.0,1.0,0,2,0,0
1,12347.0,4085.18,6.0,0.0,0.0,1.0,6.0,2,1,2,1
2,12348.0,1437.24,4.0,0.0,0.0,1.0,4.0,1,1,1,1
3,12349.0,1457.55,1.0,0.0,0.0,1.0,1.0,1,0,0,0
4,12350.0,294.40,1.0,0.0,0.0,1.0,1.0,0,1,0,0


In [125]:
df_clusters.shape

(4334, 11)

In [126]:
df_clusters.isnull().sum()

CustomerID            0
total_purchase        0
number_orders         0
number_returns        0
orders_with_gifts     0
order_per_month      35
months_with_order    35
10_12Month            0
1_3Month              0
4_6Month              0
7_9Month              0
dtype: int64

In [127]:
df_clusters.fillna(0, inplace=True)

### Customer average order value

The average order value within 1 year period.

In [129]:
df_clusters['average_order_value'] = df_clusters['total_purchase']/df_clusters['number_orders']
df_clusters.head()

,CustomerID,total_purchase,number_orders,number_returns,orders_with_gifts,order_per_month,months_with_order,10_12Month,1_3Month,4_6Month,7_9Month,average_order_value
0,12346.0,0.00,1.0,1.0,0.0,1.0,1.0,0,2,0,0,0.000000
1,12347.0,4085.18,6.0,0.0,0.0,1.0,6.0,2,1,2,1,680.863333
2,12348.0,1437.24,4.0,0.0,0.0,1.0,4.0,1,1,1,1,359.310000
3,12349.0,1457.55,1.0,0.0,0.0,1.0,1.0,1,0,0,0,1457.550000
4,12350.0,294.40,1.0,0.0,0.0,1.0,1.0,0,1,0,0,294.400000


### Recency

Determines the difference between the current date in the database and the date of the customer's last purchase.

In [132]:
current_date = df['date'].max() + pd.DateOffset(1)

recency = df_valid_orders.groupby('CustomerID')\
                         .agg({'date': lambda x: (current_date - x.max()).days})\
                         .reset_index()\
                         .rename(columns={'date':'recency'})

df_clusters = df_clusters.merge(right=recency, on='CustomerID', how='left')

In [134]:
df_clusters.isnull().sum()

CustomerID              0
total_purchase          0
number_orders           0
number_returns          0
orders_with_gifts       0
order_per_month         0
months_with_order       0
10_12Month              0
1_3Month                0
4_6Month                0
7_9Month                0
average_order_value     1
recency                35
dtype: int64

In [135]:
df_clusters.fillna(365,inplace=True)

#### Data dictionary:
- total_purchase: total amount spent within 1 year period.
- number_orders: number of orders within 1 year period.
- number_returns: number of returns registered.
- orders_with_gifts: number of orders with products of unit price equals 0.0.
- order_per_month: average number of orders per month.
- months_with_order: number of months with orders.
- 1_3Months, 4_6Months, 7_9Months, 10_12Months: number of orders per quarter of the year.
- average_order_value: average order value within 1 year period.
- recency: difference between the current date in the database and the date of the customer's last purchase.

## Exploratory Data Analysis

In [270]:
data = df_clusters[df_clusters['number_orders']>0]
fig = px.scatter(x=np.log(data['number_orders']), y=data['total_purchase'].apply(lambda x: np.log(np.abs(1+x))))

annotations = []
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.05, y=1.0,
                                  xanchor='left', yanchor='bottom',
                                  text='Total purchase value increases as the number of orders increase.',
                                  font=dict(family='Arial',
                                            size=16, 
                                            color = 'black'),
                                  showarrow=False))


fig.update_layout(figure_layout(title_text='Relation between total purchase value and number of orders.',
                                x_label='log(number of orders)',
                                y_label='log(Total purchase)',
                               annotations=annotations))

fig.show()

In [271]:
data = df_clusters[['10_12Month', '1_3Month', '4_6Month', '7_9Month']].sum()\
                                                                      .to_frame(name='count')\
                                                                      .reset_index()\
                                                                      .sort_values(by='count')

fig = px.bar(data, x='index', y='count')

annotations = []
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.05, y=1.0,
                        xanchor='left', yanchor='bottom',
                        text='Last quarter of the year have the higher amount of sales.',
                        font=dict(family='Arial',
                                  size=16, 
                                  color = 'black'),
                        showarrow=False))

fig.update_layout(figure_layout(annotations=annotations,
                                title_text='Purchase distribution over 1 year.',
                                x_label='',
                                y_label='Frequency'))

fig.show()

In [276]:
data = df_clusters[df_clusters['number_returns']>0]
fig = px.scatter(x=data['number_returns'], y=data['total_purchase'])

annotations = []
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.05, y=1.0,
                                  xanchor='left', yanchor='bottom',
                                  text='There is not an explicit relation between returns and purchase value.',
                                  font=dict(family='Arial',
                                            size=16, 
                                            color = 'black'),
                                  showarrow=False))


fig.update_layout(figure_layout(title_text='Relation between total purchase value and number of returns.',
                                x_label='Returns',
                                y_label='Purchase Value',
                               annotations=annotations))

fig.show()

In [218]:
fig = px.histogram(x=df_clusters['total_purchase'].apply(lambda x: np.sign(x)*np.log(np.abs(1+x))))

fig.update_layout(figure_layout(title_text='Distribution of Purchase Value.',
                                x_label='log(Total purchase) (£)',
                                y_label='Frequency'))
fig.show()

## Business Metrics

### Reapeat Purchase Rate, Purchase Frequency, Average Order Value metrics

Repeat Purchase Rate (RPR), Purchase Frequency (PF), Time between purchases (TBP), Average Order Value (AOV), and Customer Lifetime Value (CLV) are relevant metrics used to evaluate the business as a whole. Track such metrics helps to find new ways to grow the profitability of the business. To achieve that, businesses can focus on getting people to buy more during each visit (average order value) or to shop more often (purchase frequency). 

A loyalty program can help with both average order value and purchase frequency. The enrollment in a loyalty program encourages a customer to shop again instead of choosing a competitor. Once a shopper is enrolled strategies can be used to motivate them to shop more frequently (increasing purchase frequency).

https://blog.smile.io/how-to-calculate-purchase-frequency/

### Reapeat Purchase Rate (RPR)

Repeat purchase rate is the percentage of customers who purchase again after their first purchase. This basically measures the customer retention and is an indicator of the value provided for the customers.

In [136]:
return_customers = df_clusters[df_clusters['number_orders']>1]['CustomerID'].shape[0]
total_customers = df_clusters['CustomerID'].nunique()

RPR = return_customers*100 / total_customers
print('RPR = {:.2f}%.'.format(RPR))

RPR = 63.73%.


### Purchase Frequency (PF)

The average number of times a customer makes a purchase within a year.

In [137]:
number_of_orders = df_clusters['number_orders'].sum()
number_of_unique_customers = df_clusters.shape[0]

PF = number_of_orders / number_of_unique_customers
print('PF = {:.2f}.'.format(PF, 0))

PF = 4.11.


### Time between purchases (TBP)

Shows how often a typical customer goes before making a repeat purchase. This is a good stat to know because it allows to tailor email marketing campaigns.

In [138]:
time_period = (df_valid_orders['date'].max() - df_valid_orders['date'].min()).days

TBP = time_period / PF
print('TBP = {:.2f} days.'.format(TBP))

TBP = 88.82 days.


64% of customers purchase at least a second time and, on average, they purchase 4 times within a year and take, on average, 88 days to make a purchase. These are some of the metrics the business can track to evaluate the performance of the loyalty program. It is expected the purchase frequency to increase and the time between purchases to decrease as the program evolves. Furthermore, it might be also expected that other customers engage with the program over time increasing the repeat customer rate. 

### Average order value (AOV)

The average amount customers spend each time they make a purchase. This is a very important metric beacuse it has a strong correlation to an increase in profit. Based on this metric, businesses can determine their retention plan or marketing strategy. It is calculated dividing the yearly/monthly revenue by the number of orders your store has processed.

In [163]:
total_revenue = df['total_purchase'].sum()

yearly_AOV = total_revenue / number_of_orders
print('Yearly AOV = £{:.2f}'.format(yearly_AOV))

Yearly AOV = £445.38


This metric is usually calculated monthly. So lets calculate AOV for each month in the data set:

In [161]:
purchase_month = df.groupby('month')['total_purchase'].sum()
order_month = df_valid_orders.groupby('month')['InvoiceNo'].nunique()

monthly_AOV = purchase_month / order_month

In [168]:
fig = go.Figure()

labels = ['Monthly AOV', 'Yearly AOV']
colors = ['blue', 'gray']
line_size = [2,3]
dash = [None, 'dash']

x_data = monthly_AOV.index.to_list()
y_data = np.array([monthly_AOV, [yearly_AOV]*12])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data, y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i], dash=dash[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[-1], x_data[-1]],
        y=[y_data[i][-1], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i])
    ))


annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=1.15, y=y_trace[-1],
                                  xanchor='right', yanchor='middle',
                                  text=label,
                                  font=dict(family='Arial',
                                            size=16, 
                                            color = color),
                                  showarrow=False))


fig.update_layout(figure_layout(annotations=annotations,
                                title_text='Average Order Value (AOV) during one year period.',
                                x_label='Month',
                                y_label='AOV (£)'))
    
fig.show()

### Customer Lifetime Value

Customer lifetime value (CLV) is a measure of the average customer’s revenue generated over their entire relationship with a company.

CLV = Average Order Value x Purchase Frequency (yearly purchases) x Retention Period 

Reference: https://www.netsuite.com/portal/resource/articles/ecommerce/customer-lifetime-value-clv.shtml
https://clevertap.com/blog/customer-lifetime-value/
https://clevertap.com/blog/customer-lifetime-value/

In [150]:
# Average spends per visit (yearly_AOV) * purchase frequency (4 times per year)
CLV = yearly_AOV * PF * 1
print('Customer Lifetime Value, CLV, = £{:.2f} in one year'.format(CLV))

Customer Lifetime Value, CLV, = £1830.32 in one year


Considering the metrics calculated, it is fair to consider that customers who spent more than £1929.47 and bought at least 4 times a year are eligible for the loyalty program.

### Who are the customers that contribute more with the revenue

In [200]:
# Preparing data
plot_data = df_clusters.sort_values(by='total_purchase',ascending=False)['total_purchase'].to_frame().copy()
plot_data['contrib_revenue'] = plot_data['total_purchase'] / plot_data['total_purchase'].sum()
plot_data['cumsum'] = plot_data['contrib_revenue'].cumsum()
x_data = np.arange(0,1.1,0.1)
y_data = plot_data.quantile(q=x_data)['cumsum']
y_data.loc[0] = 0

# marker
x_marker = plot_data[plot_data['cumsum'] <= 0.80].shape[0] / df_clusters.shape[0]
y_marker = 0.8

fig = px.line(x=x_data,
              y=y_data)

fig.add_trace(go.Scatter(
        x=[x_marker],
        y=[y_marker],
        mode='markers',
        marker=dict(color='blue',size=10)))

annotations = []
annotations.append(dict(xref='paper', x=x_marker+0.01, y=y_marker-0.02,
                                  xanchor='left', yanchor='middle',
                                  text='{:.0f}% of the revenue comes from {:.0f}% of the customers.'\
                                             .format(y_marker*100,x_marker*100),
                                  font=dict(family='Arial',
                                            size=16, 
                                            color = 'blue'),
                                  showarrow=False))

fig.update_layout(figure_layout(annotations=annotations,
                                title_text='Customer Contribution to the UK Retailer Revenue',
                                x_label='Proportion of customers',
                                y_label='Proportion of Revenue'))

fig.show()

To be continued...

https://towardsdatascience.com/recency-frequency-monetary-model-with-python-and-how-sephora-uses-it-to-optimize-their-google-d6a0707c5f17

https://iterable.com/blog/the-secret-to-customer-lifetime-value-rfm/

https://www.ibm.com/docs/en/spss-statistics/27.0.0?topic=marketing-rfm-analysis

https://clevertap.com/blog/rfm-analysis/

https://towardsdatascience.com/rfmt-segmentation-using-k-means-clustering-76bc5040ead5

https://www.omniconvert.com/blog/what-is-repeat-customer-how-to-identify/